# 第6章: 高度な統計解析

この章では、より高度な統計手法を学びます。

## 学習目標
- 仮説検定の基礎
- t検定
- カイ二乗検定
- ベイズ分類器
- k-means クラスタリング
- 時系列分析の基礎

In [ ]:
// ライブラリの読み込み
const math = await import('https://esm.sh/mathjs@13.0.0');
const ss = await import('https://esm.sh/simple-statistics@7.8.8');
console.log('Libraries loaded!');

## 6.1 仮説検定の基礎

### 6.1.1 仮説検定の考え方

In [ ]:
console.log('=== 仮説検定の基本概念 ===');
console.log('');
console.log('【手順】');
console.log('1. 帰無仮説 (H₀) と対立仮説 (H₁) を設定');
console.log('2. 有意水準 α を決める（通常 0.05 または 0.01）');
console.log('3. 検定統計量を計算');
console.log('4. p値を計算');
console.log('5. p値 < α なら H₀ を棄却');
console.log('');
console.log('【エラーの種類】');
console.log('第1種の過誤（α）: H₀が真なのに棄却する誤り');
console.log('第2種の過誤（β）: H₀が偽なのに棄却しない誤り');
console.log('');
console.log('検出力（Power）= 1 - β');

### 6.1.2 z検定（母平均の検定）

In [ ]:
// z検定の例
// ある工場の製品の重量が規定値（500g）と異なるか検定

const specification = 500;  // 規定値
const populationStd = 10;   // 母標準偏差（既知）
const sampleMean = 498;     // 標本平均
const sampleSize = 36;      // サンプルサイズ
const alpha = 0.05;         // 有意水準

console.log('=== z検定: 母平均の検定 ===');
console.log('');
console.log('【問題設定】');
console.log('規定値 μ₀ =', specification, 'g');
console.log('母標準偏差 σ =', populationStd, 'g（既知）');
console.log('標本平均 x̄ =', sampleMean, 'g');
console.log('サンプルサイズ n =', sampleSize);
console.log('');

console.log('【仮説】');
console.log('H₀: μ = 500 (重量は規定通り)');
console.log('H₁: μ ≠ 500 (重量は規定と異なる) [両側検定]');
console.log('');

// z統計量の計算
// z = (x̄ - μ₀) / (σ / √n)
const standardError = populationStd / Math.sqrt(sampleSize);
const zStatistic = (sampleMean - specification) / standardError;

console.log('【計算】');
console.log('標準誤差 SE =', standardError.toFixed(4));
console.log('z統計量 =', zStatistic.toFixed(4));

// p値の計算（両側検定）
const pValue = 2 * (1 - ss.cumulativeStdNormalProbability(Math.abs(zStatistic)));
console.log('p値 =', pValue.toFixed(4));
console.log('');

// 判定
console.log('【判定】');
console.log('有意水準 α =', alpha);
if (pValue < alpha) {
    console.log('p値 < α なので、H₀を棄却');
    console.log('→ 重量は規定値と有意に異なる');
} else {
    console.log('p値 ≥ α なので、H₀を棄却できない');
    console.log('→ 重量が規定値と異なるとは言えない');
}

## 6.2 t検定

### 6.2.1 1標本t検定

In [ ]:
// 1標本t検定
// 新しい学習法で平均点が70点より高くなったか検定

const benchmarkScore = 70;
const studentScores = [72, 78, 65, 80, 85, 75, 70, 82, 77, 73, 68, 88];

console.log('=== 1標本t検定 ===');
console.log('');
console.log('【問題設定】');
console.log('基準値 μ₀ =', benchmarkScore, '点');
console.log('データ:', studentScores);
console.log('');

const n = studentScores.length;
const xBar = ss.mean(studentScores);
const s = ss.sampleStandardDeviation(studentScores);
const df = n - 1;

console.log('標本平均 x̄ =', xBar.toFixed(2));
console.log('標本標準偏差 s =', s.toFixed(4));
console.log('サンプルサイズ n =', n);
console.log('自由度 df =', df);
console.log('');

console.log('【仮説】');
console.log('H₀: μ = 70');
console.log('H₁: μ > 70 [片側検定]');
console.log('');

// t統計量
const tStatistic = (xBar - benchmarkScore) / (s / Math.sqrt(n));
console.log('【計算】');
console.log('t統計量 =', tStatistic.toFixed(4));

// simple-statisticsのtTest
const tTestResult = ss.tTestTwoSample(studentScores, [benchmarkScore], { difference: 0 });
console.log('');
console.log('t検定結果（片側検定のp値の目安）:');
console.log('t =', tStatistic.toFixed(4));
console.log('');

// 臨界値との比較（α = 0.05、片側）
const criticalT = 1.796;  // df=11, α=0.05, 片側
console.log('【判定】');
console.log('臨界値 t₀.₀₅(11) ≈', criticalT);
if (Math.abs(tStatistic) > criticalT) {
    console.log('|t| > 臨界値 なので、H₀を棄却');
    console.log('→ 新しい学習法は有意に効果がある');
} else {
    console.log('|t| ≤ 臨界値 なので、H₀を棄却できない');
    console.log('→ 効果があるとは言えない');
}

### 6.2.2 2標本t検定（独立2群）

In [ ]:
// 2標本t検定
// 2つのグループの平均に差があるか検定

const groupA = [85, 78, 92, 88, 76, 82, 90, 85, 79, 87];
const groupB = [72, 68, 75, 70, 78, 65, 74, 71, 69, 73];

console.log('=== 2標本t検定（独立2群） ===');
console.log('');
console.log('グループA:', groupA);
console.log('グループB:', groupB);
console.log('');

const nA = groupA.length;
const nB = groupB.length;
const meanA = ss.mean(groupA);
const meanB = ss.mean(groupB);
const varA = ss.sampleVariance(groupA);
const varB = ss.sampleVariance(groupB);

console.log('グループA: n=' + nA + ', 平均=' + meanA.toFixed(2) + ', 分散=' + varA.toFixed(2));
console.log('グループB: n=' + nB + ', 平均=' + meanB.toFixed(2) + ', 分散=' + varB.toFixed(2));
console.log('');

// Welchのt検定（等分散を仮定しない）
const seDiff = Math.sqrt(varA / nA + varB / nB);
const tStat = (meanA - meanB) / seDiff;

// Welch-Satterthwaiteの自由度
const dfWelch = Math.pow(varA / nA + varB / nB, 2) / 
    (Math.pow(varA / nA, 2) / (nA - 1) + Math.pow(varB / nB, 2) / (nB - 1));

console.log('【Welchのt検定】');
console.log('t統計量 =', tStat.toFixed(4));
console.log('自由度 ≈', dfWelch.toFixed(2));

// simple-statisticsでのt検定
const tTestTwoSample = ss.tTestTwoSample(groupA, groupB);
console.log('');
console.log('simple-statistics t検定 p値 ≈', tTestTwoSample ? tTestTwoSample.toFixed(4) : 'N/A');

console.log('');
console.log('【判定】');
if (Math.abs(tStat) > 2.1)  {  // 近似的な臨界値
    console.log('2群の平均には有意な差がある');
    console.log('グループAの方が', (meanA - meanB).toFixed(2), '点高い');
} else {
    console.log('2群の平均に有意な差があるとは言えない');
}

### 6.2.3 対応のあるt検定

In [ ]:
// 対応のあるt検定
// 同じ被験者の事前・事後の変化を検定

const before = [65, 70, 72, 68, 75, 60, 78, 71, 69, 74];
const after = [70, 75, 78, 72, 82, 68, 85, 76, 75, 80];

console.log('=== 対応のあるt検定 ===');
console.log('');
console.log('事前:', before);
console.log('事後:', after);
console.log('');

// 差を計算
const differences = before.map((b, i) => after[i] - b);
console.log('差 (事後 - 事前):', differences);

const meanDiff = ss.mean(differences);
const stdDiff = ss.sampleStandardDeviation(differences);
const nDiff = differences.length;

console.log('');
console.log('差の平均 d̄ =', meanDiff.toFixed(2));
console.log('差の標準偏差 s_d =', stdDiff.toFixed(4));
console.log('');

// t統計量
const tPaired = meanDiff / (stdDiff / Math.sqrt(nDiff));
console.log('【仮説】');
console.log('H₀: μ_d = 0 (変化なし)');
console.log('H₁: μ_d ≠ 0 (変化あり)');
console.log('');
console.log('t統計量 =', tPaired.toFixed(4));
console.log('自由度 =', nDiff - 1);

console.log('');
console.log('【判定】');
const criticalTPaired = 2.262;  // df=9, α=0.05, 両側
console.log('臨界値 t₀.₀₂₅(9) ≈', criticalTPaired);
if (Math.abs(tPaired) > criticalTPaired) {
    console.log('|t| > 臨界値 なので、H₀を棄却');
    console.log('→ 事前・事後で有意な変化がある');
    console.log('平均して', meanDiff.toFixed(2), '点上昇');
} else {
    console.log('有意な変化があるとは言えない');
}

## 6.3 カイ二乗検定

### 6.3.1 適合度検定

In [ ]:
// カイ二乗適合度検定
// サイコロが公平かどうか検定

const observed = [18, 22, 15, 20, 25, 20];  // 観測度数（120回振った結果）
const total = observed.reduce((a, b) => a + b, 0);
const expected = observed.map(() => total / 6);  // 期待度数（均等）

console.log('=== カイ二乗適合度検定 ===');
console.log('');
console.log('【サイコロを120回振った結果】');
console.log('出目:     [1,  2,  3,  4,  5,  6]');
console.log('観測度数:', observed);
console.log('期待度数:', expected);
console.log('');

console.log('【仮説】');
console.log('H₀: サイコロは公平（各目の出る確率は1/6）');
console.log('H₁: サイコロは公平でない');
console.log('');

// カイ二乗統計量 = Σ (O - E)² / E
let chiSquared = 0;
console.log('計算過程:');
observed.forEach((o, i) => {
    const e = expected[i];
    const contrib = Math.pow(o - e, 2) / e;
    chiSquared += contrib;
    console.log(`  目${i+1}: (${o} - ${e})² / ${e} = ${contrib.toFixed(4)}`);
});

console.log('');
console.log('χ² =', chiSquared.toFixed(4));
console.log('自由度 df =', observed.length - 1);

// 判定（臨界値 χ²₀.₀₅(5) = 11.07）
const criticalChiSq = 11.07;
console.log('');
console.log('【判定】');
console.log('臨界値 χ²₀.₀₅(5) =', criticalChiSq);
if (chiSquared > criticalChiSq) {
    console.log('χ² > 臨界値 なので、H₀を棄却');
    console.log('→ サイコロは公平でない可能性がある');
} else {
    console.log('χ² ≤ 臨界値 なので、H₀を棄却できない');
    console.log('→ サイコロが公平でないとは言えない');
}

### 6.3.2 独立性の検定

In [ ]:
// カイ二乗独立性検定
// 性別と商品選好の関連を検定

// 分割表（クロス集計表）
const contingencyTable = [
    [30, 20, 10],  // 男性: 商品A, B, C
    [20, 25, 25]   // 女性: 商品A, B, C
];

console.log('=== カイ二乗独立性検定 ===');
console.log('');
console.log('【分割表】');
console.log('         商品A  商品B  商品C  計');
console.log('男性      30     20     10    60');
console.log('女性      20     25     25    70');
console.log('計        50     45     35   130');
console.log('');

console.log('【仮説】');
console.log('H₀: 性別と商品選好は独立');
console.log('H₁: 性別と商品選好は独立でない');
console.log('');

// 周辺度数の計算
const rowTotals = contingencyTable.map(row => row.reduce((a, b) => a + b, 0));
const colTotals = [0, 0, 0];
contingencyTable.forEach(row => {
    row.forEach((cell, j) => colTotals[j] += cell);
});
const grandTotal = rowTotals.reduce((a, b) => a + b, 0);

// 期待度数の計算
console.log('期待度数:');
let chiSquaredInd = 0;
contingencyTable.forEach((row, i) => {
    const expectedRow = [];
    row.forEach((observed, j) => {
        const expected = (rowTotals[i] * colTotals[j]) / grandTotal;
        expectedRow.push(expected.toFixed(2));
        chiSquaredInd += Math.pow(observed - expected, 2) / expected;
    });
    const gender = i === 0 ? '男性' : '女性';
    console.log(`  ${gender}: [${expectedRow.join(', ')}]`);
});

console.log('');
console.log('χ² =', chiSquaredInd.toFixed(4));

// 自由度 = (行数-1) × (列数-1)
const dfInd = (contingencyTable.length - 1) * (contingencyTable[0].length - 1);
console.log('自由度 df =', dfInd);

// 判定（臨界値 χ²₀.₀₅(2) = 5.991）
const criticalChiSqInd = 5.991;
console.log('');
console.log('【判定】');
console.log('臨界値 χ²₀.₀₅(2) =', criticalChiSqInd);
if (chiSquaredInd > criticalChiSqInd) {
    console.log('χ² > 臨界値 なので、H₀を棄却');
    console.log('→ 性別と商品選好には関連がある');
} else {
    console.log('χ² ≤ 臨界値 なので、H₀を棄却できない');
    console.log('→ 性別と商品選好に関連があるとは言えない');
}

## 6.4 ベイズ分類器

In [ ]:
// 単純ベイズ分類器
console.log('=== 単純ベイズ分類器 ===');
console.log('');

// simple-statisticsのベイズ分類器
const bayes = new ss.BayesianClassifier();

// 訓練データ（スパムメール分類の例）
// 特徴: 単語の出現
bayes.train({ free: 1, money: 1, click: 1 }, 'spam');
bayes.train({ free: 1, offer: 1, limited: 1 }, 'spam');
bayes.train({ money: 1, win: 1, prize: 1 }, 'spam');
bayes.train({ meeting: 1, schedule: 1, project: 1 }, 'ham');
bayes.train({ report: 1, deadline: 1, review: 1 }, 'ham');
bayes.train({ meeting: 1, tomorrow: 1, agenda: 1 }, 'ham');

console.log('訓練データ:');
console.log('  spam: "free money click", "free offer limited", "money win prize"');
console.log('  ham: "meeting schedule project", "report deadline review", "meeting tomorrow agenda"');
console.log('');

// 分類
const testCases = [
    { free: 1, money: 1 },
    { meeting: 1, project: 1 },
    { free: 1, schedule: 1 }
];

console.log('分類結果:');
testCases.forEach((test, i) => {
    const result = bayes.score(test);
    const features = Object.keys(test).join(', ');
    console.log(`  テスト${i+1} [${features}]:`);
    console.log(`    spam確率: ${result.spam ? result.spam.toFixed(4) : 'N/A'}`);
    console.log(`    ham確率: ${result.ham ? result.ham.toFixed(4) : 'N/A'}`);
});

## 6.5 k-means クラスタリング

In [ ]:
// k-means クラスタリングの実装
console.log('=== k-means クラスタリング ===');
console.log('');

// 2次元データ
const clusterData = [
    [1, 2], [1.5, 1.8], [2, 1.5], [2.5, 2.5],  // クラスタ1
    [6, 5], [6.5, 5.5], [7, 5], [7.5, 6],      // クラスタ2
    [1, 6], [1.5, 6.5], [2, 7], [2.5, 6.5]     // クラスタ3
];

console.log('データ数:', clusterData.length);
console.log('');

// simple-statisticsのk-means
const k = 3;
const clusters = ss.ckmeans(clusterData.map(d => d[0] + d[1] * 10), k);

// 独自のk-means実装
function kMeans(data, k, maxIterations = 100) {
    // 初期中心をランダムに選択
    let centroids = [];
    const shuffled = [...data].sort(() => Math.random() - 0.5);
    for (let i = 0; i < k; i++) {
        centroids.push([...shuffled[i]]);
    }
    
    let assignments = new Array(data.length).fill(0);
    
    for (let iter = 0; iter < maxIterations; iter++) {
        // 各点を最も近い中心に割り当て
        const newAssignments = data.map(point => {
            let minDist = Infinity;
            let minIdx = 0;
            centroids.forEach((centroid, idx) => {
                const dist = Math.sqrt(
                    Math.pow(point[0] - centroid[0], 2) +
                    Math.pow(point[1] - centroid[1], 2)
                );
                if (dist < minDist) {
                    minDist = dist;
                    minIdx = idx;
                }
            });
            return minIdx;
        });
        
        // 収束チェック
        if (JSON.stringify(assignments) === JSON.stringify(newAssignments)) {
            console.log('収束しました（反復回数:', iter + 1, '）');
            break;
        }
        assignments = newAssignments;
        
        // 中心を更新
        for (let c = 0; c < k; c++) {
            const clusterPoints = data.filter((_, i) => assignments[i] === c);
            if (clusterPoints.length > 0) {
                centroids[c] = [
                    ss.mean(clusterPoints.map(p => p[0])),
                    ss.mean(clusterPoints.map(p => p[1]))
                ];
            }
        }
    }
    
    return { assignments, centroids };
}

const result = kMeans(clusterData, 3);

console.log('');
console.log('クラスタリング結果:');
result.centroids.forEach((centroid, i) => {
    const members = clusterData.filter((_, j) => result.assignments[j] === i);
    console.log(`クラスタ${i + 1}:`);
    console.log(`  中心: (${centroid[0].toFixed(2)}, ${centroid[1].toFixed(2)})`);
    console.log(`  メンバー数: ${members.length}`);
    console.log(`  メンバー:`, members);
});

## 6.6 時系列分析の基礎

### 6.6.1 移動平均

In [ ]:
// 時系列データ（月間売上）
const monthlySales = [
    100, 120, 115, 130, 125, 140, 
    145, 160, 155, 170, 180, 190,
    185, 200, 210, 205, 220, 230
];

console.log('=== 移動平均 ===');
console.log('');
console.log('月間売上データ:', monthlySales);
console.log('');

// 単純移動平均
function simpleMovingAverage(data, windowSize) {
    const result = [];
    for (let i = windowSize - 1; i < data.length; i++) {
        const window = data.slice(i - windowSize + 1, i + 1);
        result.push(ss.mean(window));
    }
    return result;
}

const sma3 = simpleMovingAverage(monthlySales, 3);
const sma5 = simpleMovingAverage(monthlySales, 5);

console.log('3期移動平均:', sma3.map(v => v.toFixed(1)));
console.log('5期移動平均:', sma5.map(v => v.toFixed(1)));

### 6.6.2 指数移動平均

In [ ]:
// 指数移動平均（EMA）
function exponentialMovingAverage(data, alpha) {
    const result = [data[0]];
    for (let i = 1; i < data.length; i++) {
        const ema = alpha * data[i] + (1 - alpha) * result[i - 1];
        result.push(ema);
    }
    return result;
}

console.log('=== 指数移動平均 ===');
console.log('');

const ema02 = exponentialMovingAverage(monthlySales, 0.2);
const ema05 = exponentialMovingAverage(monthlySales, 0.5);

console.log('α=0.2 の指数移動平均:', ema02.map(v => v.toFixed(1)));
console.log('α=0.5 の指数移動平均:', ema05.map(v => v.toFixed(1)));
console.log('');
console.log('注: αが大きいほど直近のデータに重みを置く');

### 6.6.3 トレンド分析

In [ ]:
// 線形トレンドの推定
console.log('=== トレンド分析 ===');
console.log('');

// 時点を説明変数とした回帰
const timePoints = monthlySales.map((_, i) => i + 1);
const trendReg = ss.linearRegression(
    timePoints.map((t, i) => [t, monthlySales[i]])
);

console.log('線形トレンド:');
console.log(`  売上 = ${trendReg.b.toFixed(2)} + ${trendReg.m.toFixed(2)} × 時点`);
console.log('');

// 予測
const trendLine = ss.linearRegressionLine(trendReg);
console.log('将来予測:');
console.log('  19ヶ月目:', trendLine(19).toFixed(1));
console.log('  20ヶ月目:', trendLine(20).toFixed(1));
console.log('  24ヶ月目:', trendLine(24).toFixed(1));

// トレンドの強さ
const trendRSquared = ss.rSquared(
    timePoints.map((t, i) => [t, monthlySales[i]]),
    trendLine
);
console.log('');
console.log('トレンドのR²:', trendRSquared.toFixed(4));
console.log('→ 売上は月平均約', trendReg.m.toFixed(2), '増加している');

## 6.7 実践例: A/Bテスト

In [ ]:
// A/Bテストの分析
console.log('=== A/Bテスト分析 ===');
console.log('');

// シナリオ: 新しいボタンデザインでコンバージョン率が上がるか
const controlGroup = {
    visitors: 1000,
    conversions: 45
};

const treatmentGroup = {
    visitors: 1000,
    conversions: 62
};

console.log('【データ】');
console.log('コントロール群（従来デザイン）:');
console.log('  訪問者:', controlGroup.visitors);
console.log('  コンバージョン:', controlGroup.conversions);
console.log('  率:', (controlGroup.conversions / controlGroup.visitors * 100).toFixed(2) + '%');
console.log('');
console.log('トリートメント群（新デザイン）:');
console.log('  訪問者:', treatmentGroup.visitors);
console.log('  コンバージョン:', treatmentGroup.conversions);
console.log('  率:', (treatmentGroup.conversions / treatmentGroup.visitors * 100).toFixed(2) + '%');
console.log('');

// 2標本比率の検定（z検定）
const p1 = controlGroup.conversions / controlGroup.visitors;
const p2 = treatmentGroup.conversions / treatmentGroup.visitors;
const n1 = controlGroup.visitors;
const n2 = treatmentGroup.visitors;

// プールした比率
const pooledP = (controlGroup.conversions + treatmentGroup.conversions) / (n1 + n2);

// z統計量
const sePooled = Math.sqrt(pooledP * (1 - pooledP) * (1/n1 + 1/n2));
const zAB = (p2 - p1) / sePooled;

console.log('【検定】');
console.log('H₀: p₁ = p₂ (コンバージョン率に差がない)');
console.log('H₁: p₂ > p₁ (新デザインの方が高い) [片側検定]');
console.log('');
console.log('z統計量:', zAB.toFixed(4));

// p値（片側）
const pValueAB = 1 - ss.cumulativeStdNormalProbability(zAB);
console.log('p値（片側）:', pValueAB.toFixed(4));
console.log('');

console.log('【判定】');
if (pValueAB < 0.05) {
    console.log('p値 < 0.05 なので、有意水準5%で H₀を棄却');
    console.log('→ 新デザインは有意にコンバージョン率が高い');
    
    // 効果量
    const lift = (p2 - p1) / p1 * 100;
    console.log('');
    console.log('効果量（リフト）:', lift.toFixed(1) + '%');
} else {
    console.log('p値 ≥ 0.05 なので、H₀を棄却できない');
    console.log('→ 有意な差があるとは言えない');
}

## 6.8 練習問題

### 練習1: t検定

以下の2グループのデータに対して、平均に有意差があるか検定してください。

```
グループA = [78, 82, 75, 80, 85, 79, 83, 77]
グループB = [85, 88, 90, 82, 87, 91, 86, 89]
```

In [ ]:
// ここに回答を書いてください



### 練習2: 移動平均

以下のデータに対して、3期移動平均と5期移動平均を計算してください。

```
data = [10, 15, 12, 18, 20, 22, 25, 23, 28, 30, 32, 35]
```

In [ ]:
// ここに回答を書いてください



## まとめ

この章では以下を学びました：

1. **仮説検定の基礎**: 帰無仮説、対立仮説、p値、有意水準
2. **t検定**: 1標本t検定、2標本t検定、対応のあるt検定
3. **カイ二乗検定**: 適合度検定、独立性の検定
4. **ベイズ分類器**: 単純ベイズ分類
5. **クラスタリング**: k-means法
6. **時系列分析**: 移動平均、指数移動平均、トレンド分析
7. **A/Bテスト**: 比率の検定

これで、数学・統計学チュートリアルの全章が完了しました。学んだ知識を実際のデータ分析に活かしてください！